<h1>DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [1]:
from src.StreamPort.device.DeviceEngine import DeviceEngine

#Creates an empty DeviceEngine object and prints it
dev = DeviceEngine()
dev.print()


DeviceEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-05-21 22:15:11.505426 
  analyses: 0 
  settings: 0 



<h1>DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the analysis details for each individual device.

In [2]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()


Analysis
  name: None
  replicate: None
  blank: None
  data:
  Empty

